<a href="https://githubtocolab.com/gee-community/geemap/blob/master/docs/notebooks/32_supervised_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
# !pip install geemap

# Machine Learning with Earth Engine - Supervised Classification

## Supervised classification algorithms available in Earth Engine

Source: https://developers.google.com/earth-engine/classification

The `Classifier` package handles supervised classification by traditional ML algorithms running in Earth Engine. These classifiers include CART, RandomForest, NaiveBayes and SVM. The general workflow for classification is:

1. Collect training data. Assemble features which have a property that stores the known class label and properties storing numeric values for the predictors.
2. Instantiate a classifier. Set its parameters if necessary.
3. Train the classifier using the training data.
4. Classify an image or feature collection.
5. Estimate classification error with independent validation data.

The training data is a `FeatureCollection` with a property storing the class label and properties storing predictor variables. Class labels should be consecutive, integers starting from 0. If necessary, use remap() to convert class values to consecutive integers. The predictors should be numeric.

![](https://i.imgur.com/vROsEiq.png)

## Step-by-step tutorial

### Import libraries

In [1]:
import ee
import geemap

In [4]:
ee.Authenticate()
ee.Initialize(project='ee-matthewrungaro')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


### Create an interactive map

In [5]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### Add data to the map

In [20]:
point = ee.Geometry.Point([-102.373588, 43.807540])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(point)
    .filterDate("2016-01-01", "2016-12-31")
    .sort("CLOUD_COVER")
    .first()
    .select("SR_B[1-7]")
)

vis_params = {"min": 0, "max": 3000, "bands": ["SR_B5", "SR_B4", "SR_B3"]}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

### Check image properties

In [21]:
ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo()

'2016-07-25'

In [22]:
image.get("CLOUD_COVER").getInfo()

0

### Make training dataset

There are several ways you can create a region for generating the training dataset.

- Draw a shape (e.g., rectangle) on the map and the use `region = Map.user_roi`
- Define a geometry, such as `region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])`
- Create a buffer zone around a point, such as `region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)`
- If you don't define a region, it will use the image footprint by default

In [ ]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
# region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

In this example, we are going to use the [USGS National Land Cover Database (NLCD)](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD) to create label dataset for training


![](https://i.imgur.com/7QoRXxu.png)

In [23]:
nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(image.geometry())
Map.addLayer(nlcd, {}, "NLCD")
Map

Map(bottom=24179.0, center=[43.807539999999996, -102.373588], controls=(WidgetControl(options=['position', 'tr…

In [24]:
# Make the training dataset.
points = nlcd.sample(
    **{
        "region": image.geometry(),
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)

In [25]:
print(points.size().getInfo())

5000


In [26]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-101.57198464151953, 42.449229962551684]}, 'id': '0', 'properties': {'landcover': 71}}


### Train the classifier

In [29]:
# Use these bands for prediction.
bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]


# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [28]:
print(training.first().getInfo())

EEException: Image.select: Band pattern 'B1' did not match any bands. Available bands: [SR_B1, SR_B2, SR_B3, SR_B4, SR_B5, SR_B6, SR_B7]

### Classify the image

In [30]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")
Map

Map(bottom=24179.0, center=[43.807539999999996, -102.373588], controls=(WidgetControl(options=['position', 'tr…

### Render categorical map

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps.

In [31]:
class_values = nlcd.get("landcover_class_values").getInfo()
class_values

[11,
 12,
 21,
 22,
 23,
 24,
 31,
 41,
 42,
 43,
 51,
 52,
 71,
 72,
 73,
 74,
 81,
 82,
 90,
 95]

In [32]:
class_palette = nlcd.get("landcover_class_palette").getInfo()
class_palette

['476ba1',
 'd1defa',
 'decaca',
 'd99482',
 'ee0000',
 'ab0000',
 'b3aea3',
 '68ab63',
 '1c6330',
 'b5ca8f',
 'a68c30',
 'ccba7d',
 'e3e3c2',
 'caca78',
 '99c247',
 '78ae94',
 'dcd93d',
 'ab7028',
 'bad9eb',
 '70a3ba']

In [33]:
landcover = result.set("classification_class_values", class_values)
landcover = landcover.set("classification_class_palette", class_palette)

In [34]:
Map.addLayer(landcover, {}, "Land cover")
Map

Map(bottom=24378.003805818225, center=[43.01266559990569, -101.6182204898963], controls=(WidgetControl(options…

### Visualize the result

In [35]:
print("Change layer opacity:")
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

Change layer opacity:


Box(children=(FloatSlider(value=1.0, description='opacity', max=1.0),))

### Add a legend to the map

In [36]:
Map.add_legend(builtin_legend="NLCD")
Map

Map(bottom=24378.0, center=[43.01266559990569, -101.6182204898963], controls=(WidgetControl(options=['position…

### Export the result

Export the result directly to your computer:

In [ ]:
import os

out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
out_file = os.path.join(out_dir, "landcover.tif")

In [ ]:
geemap.ee_export_image(landcover, filename=out_file, scale=900)

Export the result to Google Drive:

In [ ]:
geemap.ee_export_image_to_drive(
    landcover, description="landcover", folder="export", scale=900
)